In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

In [3]:
train_data = train_data.drop(columns=['Name','Cabin'])
train_data['family_member'] = train_data['SibSp'] + train_data['Parch']
train_data = train_data.drop(columns=['SibSp', 'Parch'])

In [4]:
test_data = test_data.drop(columns=['Name','Cabin'])
test_data['family_member'] = test_data['SibSp'] + test_data['Parch']
test_data = test_data.drop(columns=['SibSp', 'Parch'])

Percentage of column ith missing 

**Remove Cabin as it contains many Nan Data**

In [5]:
train_data.fillna(0,inplace=True)
test_data.fillna(0,inplace=True)

**Splitting labels and target**

In [6]:
X = train_data.drop(columns=['Survived'])
Y = train_data['Survived']



#choose the features we want to train, just forget the float data
cate_features_index = np.where(X.dtypes != float)[0]


In [7]:
from catboost import CatBoostClassifier, cv, Pool

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
#make the x for train and test (also called validation data) 
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,train_size=0.82,random_state=42)

In [10]:
clf =CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42)

**kfOLD validation**

In [11]:
#now just to make the model to fit the data
clf.fit(xtrain,ytrain,cat_features=cate_features_index,eval_set=(xtest,ytest), early_stopping_rounds=50)

Learning rate set to 0.029319
0:	learn: 0.8068493	test: 0.8012422	best: 0.8012422 (0)	total: 60.6ms	remaining: 1m
1:	learn: 0.8068493	test: 0.8074534	best: 0.8074534 (1)	total: 65.4ms	remaining: 32.6s
2:	learn: 0.8109589	test: 0.8074534	best: 0.8074534 (1)	total: 67.3ms	remaining: 22.4s
3:	learn: 0.8109589	test: 0.8012422	best: 0.8074534 (1)	total: 70.5ms	remaining: 17.6s
4:	learn: 0.8109589	test: 0.8012422	best: 0.8074534 (1)	total: 74.3ms	remaining: 14.8s
5:	learn: 0.8082192	test: 0.8012422	best: 0.8074534 (1)	total: 77.5ms	remaining: 12.8s
6:	learn: 0.8095890	test: 0.7950311	best: 0.8074534 (1)	total: 82.7ms	remaining: 11.7s
7:	learn: 0.8178082	test: 0.8074534	best: 0.8074534 (1)	total: 86.1ms	remaining: 10.7s
8:	learn: 0.8164384	test: 0.8074534	best: 0.8074534 (1)	total: 89.3ms	remaining: 9.83s
9:	learn: 0.8178082	test: 0.8136646	best: 0.8136646 (9)	total: 92.8ms	remaining: 9.19s
10:	learn: 0.8178082	test: 0.8136646	best: 0.8136646 (9)	total: 96.7ms	remaining: 8.7s
11:	learn: 0.812

**Preprocessing Test_Data like train data**

In [12]:
test_id = test_data.PassengerId

In [13]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Sex              0
Age              0
Ticket           0
Fare             0
Embarked         0
family_member    0
dtype: int64

In [14]:
prediction = clf.predict(test_data)

In [15]:
len(test_id),len(prediction)

(418, 418)

In [16]:
df_sub = pd.DataFrame()
df_sub['PassengerId'] = test_id
df_sub['Survived'] = prediction.astype(np.int)

df_sub.to_csv('gender_submission.csv', index=False)